In [ ]:
!pip install watchtower

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-8741e690-3f37-4ee8-a54c-a72a3f358cc4/bin/python -m pip install --upgrade pip' command.


### Better solution

In [ ]:
import boto3
import json
import time
import os

def create_watchtower_log(log_group_name, log_stream_name, log_message, client):

    # Create the log group if it doesn't exist
    try:
        client.create_log_group(logGroupName=log_group_name)
    except client.exceptions.ResourceAlreadyExistsException:
        pass

    # Create the log stream if it doesn't exist
    try:
        client.create_log_stream(logGroupName=log_group_name, logStreamName=log_stream_name)
    except client.exceptions.ResourceAlreadyExistsException:
        pass

    # Get the current timestamp
    timestamp = int(time.time() * 1000)

    # Prepare the log event
    log_event = {
        'timestamp': timestamp,
        'message': log_message
    }

    # Format the log event as JSON
    json_log_event = json.dumps(log_event)

    # Get the sequence token for the log stream
    response = client.describe_log_streams(
        logGroupName=log_group_name,
        logStreamNamePrefix=log_stream_name
    )
    
    sequence_token = response['logStreams'][0].get('uploadSequenceToken')

    # Put the log event to CloudWatch
    if sequence_token:
        client.put_log_events(
            logGroupName=log_group_name,
            logStreamName=log_stream_name,
            logEvents=[{
                'timestamp': timestamp,
                'message': json_log_event
            }],
            sequenceToken=sequence_token
        )
    else:
        client.put_log_events(
            logGroupName=log_group_name,
            logStreamName=log_stream_name,
            logEvents=[{
                'timestamp': timestamp,
                'message': json_log_event
            }]
        )

# Example usage
if __name__ == "__main__":

    # Create a session with your access keys
    session = boto3.Session(
        aws_access_key_id='',
        aws_secret_access_key='',
        region_name='us-west-2'
    )
    # Create a CloudWatch Logs client
    client = session.client('logs')

    log_group = 'MyLogGroup'
    log_stream = 'MyLogStream'
    #message = 'This is a test log message'
    message = data
    create_watchtower_log(log_group, log_stream, message, client)


In [ ]:
data = {
    "constraint_suggestions": [
        {
            "constraint_name": "ComplianceConstraint(Compliance('insight' has value range 'N', 'Y',`insight` IN ('N', 'Y'),None))",
            "column_name": "insight",
            "current_value": "Compliance: 1",
            "description": "'insight' has value range 'N', 'Y'",
            "suggesting_rule": "CategoricalRangeRule(com.amazon.deequ.suggestions.rules.CategoricalRangeRule$$$Lambda$4119/0x000000080197e840@74f276b0)",
            "rule_description": "If we see a categorical range for a column, we suggest an IS IN (...) constraint",
            "code_for_constraint": ".isContainedIn(\"insight\", [\"N\", \"Y\"])"
        },
        {
            "constraint_name": "CompletenessConstraint(Completeness(insight,None))",
            "column_name": "insight",
            "current_value": "Completeness: 1.0",
            "description": "'insight' is not null",
            "suggesting_rule": "CompleteIfCompleteRule()",
            "rule_description": "If a column is complete in the sample, we suggest a NOT NULL constraint",
            "code_for_constraint": ".isComplete(\"insight\")"
        },
        {
            "constraint_name": "CompletenessConstraint(Completeness(review_id,None))",
            "column_name": "review_id",
            "current_value": "Completeness: 1.0",
            "description": "'review_id' is not null",
            "suggesting_rule": "CompleteIfCompleteRule()",
            "rule_description": "If a column is complete in the sample, we suggest a NOT NULL constraint",
            "code_for_constraint": ".isComplete(\"review_id\")"
        }
    ]
}

In [ ]:
data

Out[6]: {'constraint_suggestions': [{'constraint_name': "ComplianceConstraint(Compliance('insight' has value range 'N', 'Y',`insight` IN ('N', 'Y'),None))",
   'column_name': 'insight',
   'current_value': 'Compliance: 1',
   'description': "'insight' has value range 'N', 'Y'",
   'suggesting_rule': 'CategoricalRangeRule(com.amazon.deequ.suggestions.rules.CategoricalRangeRule$$$Lambda$4119/0x000000080197e840@74f276b0)',
   'rule_description': 'If we see a categorical range for a column, we suggest an IS IN (...) constraint',
   'code_for_constraint': '.isContainedIn("insight", ["N", "Y"])'},
  {'constraint_name': 'CompletenessConstraint(Completeness(insight,None))',
   'column_name': 'insight',
   'current_value': 'Completeness: 1.0',
   'description': "'insight' is not null",
   'suggesting_rule': 'CompleteIfCompleteRule()',
   'rule_description': 'If a column is complete in the sample, we suggest a NOT NULL constraint',
   'code_for_constraint': '.isComplete("insight")'},
  {'constra